# Resumo, Teoria e Prática - Power Iteration Method
> Gil Miranda<br>
> Fontes bibliográficas:
* Bernardo Costa. (2018). <i>Notebooks de aula</i>.
* Trefethen, L. & Bau, D. (1997) <i>Numerical Linear Algebra</i>. SIAM
    
---

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Calculando autovetores
> Um dos algoritmos mais clássicos de cálculo de autovetores é "multiplicar e normalizar":
tome um vetor "qualquer" $u_0$ e aplique a matriz $A$, obtendo $w_1 = Au_0$.
Normalize $w_1$, ou seja, divida pela sua norma para obter um vetor unitário de mesma direção,
e chame-o de $u_1$.
Repita: $w_2 = Au_1$, e $u_2 = \frac{w_2}{N(w_2)}$.
E assim por diante.
Em geral (isso depende de $u_0$), a sequência dos $u_n$ converge para um autovetor $u$ correspondente
ao autovalor de $A$ de maior módulo.

Semana14-Parte1-Autovetores e autovalores Completo.ipynb - [Bernardo Freitas Paulo da Costa](http://www.im.ufrj.br/bernardofpc)

Esse metódo é conhecido como Power Iteration Method, ou Metódo das Potências. O metódo converge sob certas condições, como a matriz $\boldsymbol{A}$ ser simétrica e possuir um maior autovalor em módulo.<br>
Dada estas condições, o metódo consegue fornecer uma boa aproximação do 'maior' autovetor e seu autovalor.

## Preparativos
$\renewcommand{\blacksquare}{\texttt{Q.E.D.}}$
###### Definição: Autovalor e autovetor dominante
Seja $\{\lambda_i\}_{i \in \{1,\dots,n\}}$ autovalores de uma matriz $\boldsymbol{A} \, \scriptsize{n \times n}$.<br>
$\lambda_k \in \{\lambda_i\}_{i \in \{1,\dots,n\}}$ é dito autovalor dominante de $\boldsymbol{A}$ se:
$$
|\lambda_k| > |\lambda_i|, \forall i \in \{1,\dots,n\}\setminus \{k\}
$$
O autovetor $\boldsymbol{v_k}$ correspondente ao autovalor $\lambda_k$ dominante é dito autovetor dominante. 

###### Produto de vetor linha e sua transposta
Tomando um vetor $\boldsymbol{v} \in \mathbb{R}^n$, vamos utilizar o seguinte fato
$$
\boldsymbol{v} \cdot \boldsymbol{v^T} = ||\boldsymbol{v}||^2
$$
Onde $\odot$ denota a operação de produto de matrizes, para facilitar a notação escreveremos apenas $\boldsymbol{vv^T}$

---

Proof: $\boldsymbol{v}$ é uma matriz $\scriptsize{1 \times n}$, $\boldsymbol{v^T}$ é então uma matriz $\scriptsize{n \times 1}$, logo o produto será uma matriz $\scriptsize{1\times1}$, que pode ser visto como um escalar.<br>
Como estamos lidando com uma matriz e sua transposta, teremos a seguinte relação: seja $a_{i,j}$ o elemento associado a linha $i$ e coluna $j$ de $\boldsymbol{v}$, então este mesmo elemento estará em $b_{j,i}$, elemento de $\boldsymbol{v^T}$.<br>
Seja $u_{1,1}$ o elemento único da matriz produto $\boldsymbol{U}$ e utilizando o fato que $a_{i,j} = b_{j,i}$, temos:
$$
\boldsymbol{U} = \boldsymbol{vv^T}\\
u_{1,1} = a_{1,1} \cdot b_{1,1} + a_{1,2} \cdot b_{2,1} + \dots + a_{1,n} \cdot b_{n,1}\\
u_{1,1} = \sum_{j = 1}^{n} a_{1,j}^2 \\
\therefore u_{1,1} = ||\boldsymbol{v}||^2\\
$$
<div style="text-align: right">$\blacksquare$</div>

---
#### Calculando o autovalor - O quociente de Rayleigh
Seja $\boldsymbol{v}$ um autovetor associado a uma matriz $\boldsymbol{A}$ de transformação linear, vamos olhar para seu autovalor
$$
\boldsymbol{Av} = \lambda \boldsymbol{v}\\
\boldsymbol{v^T A v} = \boldsymbol{v^T} \lambda \boldsymbol{v} \\
\boldsymbol{v^T A v} = \lambda (\boldsymbol{v^T v}) \\
\boldsymbol{A v} = \lambda ||\boldsymbol{v}||^2 \\
\therefore
\lambda = \frac{\boldsymbol{v^T A v}}{||\boldsymbol{v}||^2}
$$

Esta aproximação, chamada de quociente de Rayleigh provem de um teorema creditado a John William Rayleigh (1842-1919). Esta fórmula será útil para estimarmos o autovalor e criar um critério de parada

In [2]:
## Implementação do algoritmo acima para estimar um valor aproximado para o autovalor associado
## ao autovetor v da matriz A

def evalue_estimate(A,v):
    norm2 = np.linalg.norm(v)**2 # Norma ao quadrado do vetor v
    r = v.T@A@v  # v^T A v
    return (r / norm2)

###### Um critério de parada

Tendo em mãos um candidato a autovetor $\boldsymbol{v}$ da matriz $\boldsymbol{A}$ como podemos verificar se este é de fato um autovetor?<br>
Bom, se for um autovetor, então a seguinte relação é verdadeira:
$$
\boldsymbol{Av} = \lambda \boldsymbol{v} \\
||\boldsymbol{Av}|| - ||\lambda \boldsymbol{v}|| = 0 \\
$$

Assumindo que sabemos quem é a matriz, e tendo o candidato a autovetor, podemos calcular seu autovalor pelo quociente de Rayleigh, caso seja realmente um autovetor então a segunda relação descrita nos fornece um bom critério de parada, como estamos lidando com metódos numéricos, assumimos uma tolerância a erro $\epsilon$

$$
||\boldsymbol{Av}|| - ||\lambda \boldsymbol{v}|| < \epsilon \\
$$

Temos agora um critério de parada para o algoritmo a ser construído

## O Algoritmo

Seja $\boldsymbol{A} \in \mathbb{R^{n\times n}}$ uma matriz que admita uma base de autovetores, vamos tomar uma matriz simétrica e que possua autovetor e autovalor dominantes, então é possível construir uma sequência que converge para o autovetor e autovalor dominante da transformação linear<br>

<br>Seja $\boldsymbol{u_0} \in \mathbb{R}$ um vetor qualquer, tomamos $\boldsymbol{x_0} = \frac{ \boldsymbol{u_0}}{||\boldsymbol{u_0}||}$, o próximo termo da sequência é dado aplicando a transformação em  $\boldsymbol{x_0}$ e normalizando

$$
\boldsymbol{x_1} = \boldsymbol{A}\frac{\boldsymbol{x_0}}{||\boldsymbol{x_0}||}\\
\vdots\\
\boldsymbol{x_{k+1}} = \boldsymbol{A}\frac{\boldsymbol{x_k}}{||\boldsymbol{x_k}||} = \boldsymbol{A^k}\frac{\boldsymbol{x_0}}{||\boldsymbol{x_0}||}\\
k \to \infty \implies \boldsymbol{x_{k+1}} \to \boldsymbol{v}
$$
Onde $\boldsymbol{v}$ é o autovetor dominante

---

Proof: Como a matriz admite base de autovetores, tomamos então esta base $\{\boldsymbol{v_1},\dots , \boldsymbol{v_n}\}$<br>
Dado um vetor $\boldsymbol{x_0}$, podemos escrever como combinação linear dos vetores da base
$$
\boldsymbol{x_k} = c_1 \boldsymbol{v_1} + c_2 \boldsymbol{v_2} + c_3 \boldsymbol{v_3} + \dots + c_n \boldsymbol{v_n}  
$$
Aplicando a transformação linear $k$ vezes e utilizando o fato que $\boldsymbol{v_i}$ é autovetor
$$
\boldsymbol{A^k} \boldsymbol{x_k} = \lambda_1^k c_1 \boldsymbol{v_1} + \lambda_2^k c_2 \boldsymbol{v_2} + \lambda_3^k c_3 \boldsymbol{v_3} + \dots + \lambda_n^k c_n \boldsymbol{v_n}  
$$
Como a soma de reais é comutativa, podemos reorganizar os termos de modo que $\lambda_1$ seja o autovalor dominante, e fatorando este autovalor teremos
$$
\boldsymbol{A^k} \boldsymbol{x_k} = \lambda_1^k \left(c_1 \boldsymbol{v_1} + \frac{\lambda_2^k}{\lambda_1^k} c_2 \boldsymbol{v_2} + \frac{\lambda_3^k}{\lambda_3^k} c_3 \boldsymbol{v_3} + \dots + \frac{\lambda_n^k}{\lambda_1^k} c_n \boldsymbol{v_n}\right)  
$$
Como $\lambda_1$ é autovalor dominante, temos: 
$$
lim_{k\to \infty} \left|\frac{\lambda_i^k}{\lambda_1^k}\right| = 0
$$
Portanto
$$
\boldsymbol{A^k} \boldsymbol{x_k} = \lambda_1^k \left(c_1 \boldsymbol{v_1} + 0 c_2 \boldsymbol{v_2} + 0 c_3 \boldsymbol{v_3} + \dots + 0 c_n \boldsymbol{v_n}\right) \\
\boldsymbol{A^k} \boldsymbol{x_k} = \lambda_1^k c_1 \boldsymbol{v_1}
$$
E
$$
\boldsymbol{x_{k+1}} = \boldsymbol{A} \frac{\boldsymbol{x_k}}{||\boldsymbol{x_k}||} = \lambda \frac{\boldsymbol{x_k}}{||\boldsymbol{x_k}||}
$$
$ \therefore \boldsymbol{x_{k+1}}$ é um autovetor com módulo igual a seu autovalor
<div style="text-align: right">$\blacksquare$</div>

---

In [3]:
# Implementação númerica
def power_iter(A, tol=1e-12): # Recebe uma matriz A quadrada e uma tolerância de erro
    n,m = np.shape(A)
    assert n==m, 'Matriz A deve ser quadrada'
    
    u = np.random.rand(n) # Gera um vetor qualquer
    control = False
    while control == False:
        u = A@u # Aplica a transformação linear
        u_norm = np.linalg.norm(u)
        u *= 1/u_norm # Normaliza
        l = evalue_estimate(A,u) # Estimador de autovalor 
        err = np.linalg.norm(A@u - l*u)
        if(err < tol): control = True
    return u,l

## Testando  o metódo

Para testar se nosso metódo está funcionando bem, podemos utilizar da função `np.linalg.eig` da biblioteca `numpy`, por ser uma biblioteca númerica muito usada, podemos depositar uma 'confiança' e usar como uma referência para testar nosso algoritmo.<br>
A ideia aqui é simples, gerar aleatoriamente uma matriz $\boldsymbol{A}$ e encontrar um autovetor e um autovalor $p\_v$ e $p\_l$ pelo metódo das potências, e utilizar `np.linalg.eig` para retornar uma lista de todos os autovetores e autovalores da matriz, após isso pegamos o maior autovalor em módulo e seu autovetor associado e comparamos com $p\_v$ e $p\_l$, dada uma tolerância de $\scriptsize{10 ^{-12}}$

In [4]:
## Gerando 500 matrizes de tamanhos diferentes e comparando com o metódo do numpy, tol = 10^-8
for i in range(500):
    n = np.random.rand(1)
    t = np.random.rand(1)*100
    n = int(n*t)
    if n == 0:
          n = 1
    A = np.random.rand(n,n) # Gera a matriz
    p_v, p_l = power_iter(A) # autovetor e autovalor dominante
    l, w = np.linalg.eig(A) # numpy
    l_i = np.argmax(l) # indice do autovalor dominante
    l = l[l_i] # autovalor dominante pelo numpy
    w = w[:,l_i] # autovetor dominante pelo numpy
    assert np.allclose(abs(w), abs(p_v), rtol=1e-8, atol=0)